# The Milwaukee Journal Sentinel and Life Expectancy

**Story:** [Race gap found in pothole patching](https://web.archive.org/web/20081223094123/http://www.jsonline.com/news/milwaukee/32580034.html)

**Author:** Keegan Kyle, Grant Smith and Ben Poston, Milwaukee Journal Sentinel

**Topics:** Census Data, Geocoding, QGIS Spatial Joins, Linear Regression

**Datasets**

* Pothole fill data from the Milwaukee [Department of Public Works](https://city.milwaukee.gov/dpw)
    - **2007-2010 POTHOLES.xls**: Pothole data, July 2007-July 2010
    - **2010-2013 POTHOLES.xls**: Pothole data, July 2010-July 2013
    - **2013-2017 POTHOLES.xls**: Pothole data, July 2013-July 2017
* **tl_2013_55_tract.zip:** 2013 census tract boundaries from the [US Census Bureau](https://www.census.gov/cgi-bin/geo/shapefiles/index.php)
* **addresses_geocoded.csv:** a large selection of addresses in Milwaukee, geocoded by [Geocod.io](https://geocod.io)
* **R12216099_SL140.csv:** ACS 2013 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table A04001, Hispanic or Latino by Race
    - `R12216099.txt` is the data dictionary
* **R12216226_SL140.csv** ACS 2013 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table A14006, 2013 Median Household income
    - Data dictionary [is here](https://www.socialexplorer.com/data/ACS2013_5yr/metadata/?ds=SE&table=A14006)

# What's the story?

We're trying to figure out if the **time it took Milwaukee to fill pot holes** is related to the racial makeup of a census tract.

# Do your imports

You'll also want to set pandas to display **up to 200 columns at a time**.

In [209]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from statsmodels.sandbox.regression.predstd import wls_prediction_std

pd.set_option('max_columns', 200)

# Create a dataframe of potholes from only 2013

The delivered multiple files to us, so we'll need to filter them down to just 2013.

**Editorial decision:** When we say "a pothole from 2013," what exactly do we mean?

In [210]:
first = pd.read_excel('data/2010-2013 POTHOLES.xls')
first.head(5)

,A,Street,EnterDt,PrintDt,ResolvDt
0,3839,N 10TH ST,2013-07-15 23:35,2013-07-16 05:46,2013-07-17 05:50
1,4900,W MELVINA ST,2013-07-15 20:05,2013-07-16 05:46,2013-07-24 16:58
2,2400,W WISCONSIN AV,2013-07-15 20:00,2013-07-16 05:56,2013-07-25 14:42
3,1800,W HAMPTON AV,2013-07-15 19:55,2013-07-16 05:46,2013-07-18 06:06
4,4718,N 19TH ST,2013-07-15 19:50,2013-07-16 05:46,2013-08-02 06:08


In [211]:
second = pd.read_excel('data/2013-2017 POTHOLES.xls')
second.head(5)

,A,Street,EnterDt,PrintDt,ResolvDt
0,7741,N 59TH ST,2017-07-15 08:55,2017-07-17 05:33,2017-07-21 04:51
1,5517,N 39TH ST,2017-07-14 22:36,2017-07-17 05:33,2017-07-25 15:29
2,8242,N GRANVILLE RD,2017-07-14 18:30,2017-07-17 05:33,2017-07-17 06:55
3,3364,W LAKEFIELD DR,2017-07-14 16:35,2017-07-17 05:22,2017-07-18 05:47
4,10700,W GOOD HOPE RD,2017-07-14 15:18,2017-07-17 05:33,2017-07-21 04:58


In [212]:
potholes = pd.concat([first,second])
potholes.shape

(82073, 5)

In [213]:
# Fix the dates where null for ResolvDt
potholes.ResolvDt.fillna(potholes.PrintDt, inplace=True)

#EntrDt will be what determines 2013 potholes
potholes['Start_Date']=pd.to_datetime(potholes['EnterDt']) 
potholes['End_Date']=pd.to_datetime(potholes['ResolvDt']) 
potholes.head(15)

,A,Street,EnterDt,PrintDt,ResolvDt,Start_Date,End_Date
0,3839,N 10TH ST,2013-07-15 23:35,2013-07-16 05:46,2013-07-17 05:50,2013-07-15 23:35:00,2013-07-17 05:50:00
1,4900,W MELVINA ST,2013-07-15 20:05,2013-07-16 05:46,2013-07-24 16:58,2013-07-15 20:05:00,2013-07-24 16:58:00
2,2400,W WISCONSIN AV,2013-07-15 20:00,2013-07-16 05:56,2013-07-25 14:42,2013-07-15 20:00:00,2013-07-25 14:42:00
3,1800,W HAMPTON AV,2013-07-15 19:55,2013-07-16 05:46,2013-07-18 06:06,2013-07-15 19:55:00,2013-07-18 06:06:00
4,4718,N 19TH ST,2013-07-15 19:50,2013-07-16 05:46,2013-08-02 06:08,2013-07-15 19:50:00,2013-08-02 06:08:00
5,4500,W CLARKE ST,2013-07-15 18:05,2013-07-16 05:56,2013-08-01 14:41,2013-07-15 18:05:00,2013-08-01 14:41:00
6,6471,N 66TH ST,2013-07-15 17:44,2013-07-16 05:46,2013-08-14 05:58,2013-07-15 17:44:00,2013-08-14 05:58:00
7,6200,W BALDWIN ST,2013-07-15 17:35,2013-07-16 05:46,2013-08-02 06:09,2013-07-15 17:35:00,2013-08-02 06:09:00
8,3013,N 74TH ST,2013-07-15 17:23,2013-07-16 05:46,2013-07-16 05:49,2013-07-15 17:23:00,2013-07-16 05:49:00
9,2606,N 7TH ST,2013-07-15 17:13,2013-07-16 05:55,2013-08-07 06:06,2013-07-15 17:13:00,2013-08-07 06:06:00


In [214]:
# filter to only 2013
potholes['Year']=pd.DatetimeIndex(potholes['Start_Date']).year
potholes = potholes[potholes['Year']==2013]
potholes.head(15)

,A,Street,EnterDt,PrintDt,ResolvDt,Start_Date,End_Date,Year
0,3839,N 10TH ST,2013-07-15 23:35,2013-07-16 05:46,2013-07-17 05:50,2013-07-15 23:35:00,2013-07-17 05:50:00,2013
1,4900,W MELVINA ST,2013-07-15 20:05,2013-07-16 05:46,2013-07-24 16:58,2013-07-15 20:05:00,2013-07-24 16:58:00,2013
2,2400,W WISCONSIN AV,2013-07-15 20:00,2013-07-16 05:56,2013-07-25 14:42,2013-07-15 20:00:00,2013-07-25 14:42:00,2013
3,1800,W HAMPTON AV,2013-07-15 19:55,2013-07-16 05:46,2013-07-18 06:06,2013-07-15 19:55:00,2013-07-18 06:06:00,2013
4,4718,N 19TH ST,2013-07-15 19:50,2013-07-16 05:46,2013-08-02 06:08,2013-07-15 19:50:00,2013-08-02 06:08:00,2013
5,4500,W CLARKE ST,2013-07-15 18:05,2013-07-16 05:56,2013-08-01 14:41,2013-07-15 18:05:00,2013-08-01 14:41:00,2013
6,6471,N 66TH ST,2013-07-15 17:44,2013-07-16 05:46,2013-08-14 05:58,2013-07-15 17:44:00,2013-08-14 05:58:00,2013
7,6200,W BALDWIN ST,2013-07-15 17:35,2013-07-16 05:46,2013-08-02 06:09,2013-07-15 17:35:00,2013-08-02 06:09:00,2013
8,3013,N 74TH ST,2013-07-15 17:23,2013-07-16 05:46,2013-07-16 05:49,2013-07-15 17:23:00,2013-07-16 05:49:00,2013
9,2606,N 7TH ST,2013-07-15 17:13,2013-07-16 05:55,2013-08-07 06:06,2013-07-15 17:13:00,2013-08-07 06:06:00,2013


## What is the maximum and minimum `EnterDt` and `ResolvDt`?

Use this to confirm that your date range is what you expected. If it isn't, take a look at what might have happened with your dataset.

* **Tip:** Missing data might be a headache

In [215]:
print("minimum", potholes.EnterDt.min(), "and maximum", potholes.EnterDt.max())

minimum 2013-01-02 06:31 and maximum 2013-12-31 17:06


In [216]:
print("minimum", potholes.End_Date.min(), "and maximum", potholes.End_Date.max())

minimum 2013-01-02 06:33:00 and maximum 2014-02-10 13:00:00


## Calculate how long it took to fill potholes in 2013

Save it into a new column.

* **Tip:** It's possible to subtract two dates

In [217]:
potholes['To_Fix'] = potholes.End_Date - potholes.Start_Date
potholes.head(15)

,A,Street,EnterDt,PrintDt,ResolvDt,Start_Date,End_Date,Year,To_Fix
0,3839,N 10TH ST,2013-07-15 23:35,2013-07-16 05:46,2013-07-17 05:50,2013-07-15 23:35:00,2013-07-17 05:50:00,2013,1 days 06:15:00
1,4900,W MELVINA ST,2013-07-15 20:05,2013-07-16 05:46,2013-07-24 16:58,2013-07-15 20:05:00,2013-07-24 16:58:00,2013,8 days 20:53:00
2,2400,W WISCONSIN AV,2013-07-15 20:00,2013-07-16 05:56,2013-07-25 14:42,2013-07-15 20:00:00,2013-07-25 14:42:00,2013,9 days 18:42:00
3,1800,W HAMPTON AV,2013-07-15 19:55,2013-07-16 05:46,2013-07-18 06:06,2013-07-15 19:55:00,2013-07-18 06:06:00,2013,2 days 10:11:00
4,4718,N 19TH ST,2013-07-15 19:50,2013-07-16 05:46,2013-08-02 06:08,2013-07-15 19:50:00,2013-08-02 06:08:00,2013,17 days 10:18:00
5,4500,W CLARKE ST,2013-07-15 18:05,2013-07-16 05:56,2013-08-01 14:41,2013-07-15 18:05:00,2013-08-01 14:41:00,2013,16 days 20:36:00
6,6471,N 66TH ST,2013-07-15 17:44,2013-07-16 05:46,2013-08-14 05:58,2013-07-15 17:44:00,2013-08-14 05:58:00,2013,29 days 12:14:00
7,6200,W BALDWIN ST,2013-07-15 17:35,2013-07-16 05:46,2013-08-02 06:09,2013-07-15 17:35:00,2013-08-02 06:09:00,2013,17 days 12:34:00
8,3013,N 74TH ST,2013-07-15 17:23,2013-07-16 05:46,2013-07-16 05:49,2013-07-15 17:23:00,2013-07-16 05:49:00,2013,0 days 12:26:00
9,2606,N 7TH ST,2013-07-15 17:13,2013-07-16 05:55,2013-08-07 06:06,2013-07-15 17:13:00,2013-08-07 06:06:00,2013,22 days 12:53:00


### Hrm, well, I think we need that difference to be an integer

If your new column isn't an integer, create _another_ column that is.

* **Tip:** Just like you might use `.str.strip()` on a string column, if your column is a datetime you can use `.dt.components` to get the days, hours, minutes, seconds, etc of the column.

In [218]:
potholes['Fixed']=potholes.To_Fix.dt.components.days
potholes.head(5)

,A,Street,EnterDt,PrintDt,ResolvDt,Start_Date,End_Date,Year,To_Fix,Fixed
0,3839,N 10TH ST,2013-07-15 23:35,2013-07-16 05:46,2013-07-17 05:50,2013-07-15 23:35:00,2013-07-17 05:50:00,2013,1 days 06:15:00,1
1,4900,W MELVINA ST,2013-07-15 20:05,2013-07-16 05:46,2013-07-24 16:58,2013-07-15 20:05:00,2013-07-24 16:58:00,2013,8 days 20:53:00,8
2,2400,W WISCONSIN AV,2013-07-15 20:00,2013-07-16 05:56,2013-07-25 14:42,2013-07-15 20:00:00,2013-07-25 14:42:00,2013,9 days 18:42:00,9
3,1800,W HAMPTON AV,2013-07-15 19:55,2013-07-16 05:46,2013-07-18 06:06,2013-07-15 19:55:00,2013-07-18 06:06:00,2013,2 days 10:11:00,2
4,4718,N 19TH ST,2013-07-15 19:50,2013-07-16 05:46,2013-08-02 06:08,2013-07-15 19:50:00,2013-08-02 06:08:00,2013,17 days 10:18:00,17


# Read in your race-related census data

* **Tip:** This will probably give you an error the first time you try, because the Census Bureau doesn't think about the rest of the world.

In [219]:
race = pd.read_csv('data/R12216099_SL140.csv')
race.head(5)

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,Geo_STATECE,Geo_STATE,Geo_COUNTY,Geo_COUSUB,Geo_PLACE,Geo_PLACESE,Geo_TRACT,Geo_BLKGRP,Geo_CONCIT,Geo_AIANHH,Geo_AIANHHFP,Geo_AIHHTLI,Geo_AITSCE,Geo_AITS,Geo_ANRC,Geo_CBSA,Geo_CSA,Geo_METDIV,Geo_MACC,Geo_MEMI,Geo_NECTA,Geo_CNECTA,Geo_NECTADIV,Geo_UA,Geo_UACP,Geo_CDCURR,Geo_SLDU,Geo_SLDL,Geo_VTD,Geo_ZCTA3,Geo_ZCTA5,Geo_SUBMCD,Geo_SDELM,Geo_SDSEC,Geo_SDUNI,Geo_UR,Geo_PCI,Geo_TAZ,Geo_UGA,Geo_BTTR,Geo_BTBG,Geo_PUMA5,Geo_PUMA1,SE_A04001_001,SE_A04001_002,SE_A04001_003,SE_A04001_004,SE_A04001_005,SE_A04001_006,SE_A04001_007,SE_A04001_008,SE_A04001_009,SE_A04001_010,SE_A04001_011,SE_A04001_012,SE_A04001_013,SE_A04001_014,SE_A04001_015,SE_A04001_016,SE_A04001_017
0,55079000101,14000US55079000101,"Census Tract 1.01, Milwaukee County, Wisconsin","Census Tract 1.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4717,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5306,5015,1343,3567,13,70,0,16,6,291,37,0,0,0,0,254,0
1,55079000102,14000US55079000102,"Census Tract 1.02, Milwaukee County, Wisconsin","Census Tract 1.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4718,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3648,3549,1034,2195,36,118,0,0,166,99,67,32,0,0,0,0,0
2,55079000201,14000US55079000201,"Census Tract 2.01, Milwaukee County, Wisconsin","Census Tract 2.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4719,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4791,4019,1185,2657,0,102,0,0,75,772,120,0,0,0,0,613,39
3,55079000202,14000US55079000202,"Census Tract 2.02, Milwaukee County, Wisconsin","Census Tract 2.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4720,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6324,6148,2687,2810,87,390,12,0,162,176,113,0,0,0,0,63,0
4,55079000301,14000US55079000301,"Census Tract 3.01, Milwaukee County, Wisconsin","Census Tract 3.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4721,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339,1339,1005,220,0,90,0,10,14,0,0,0,0,0,0,0,0


## Create new dataframe describing the racial makeup of each census tract

Each row should contain:

* The census tract number
* The percent of the population that is White
* The percent of the population that is Black
* The percent of the population that is Hispanic
* The percent of the population that is a minority (non-White)

You can either calculate and filter on the original shapefile, or create a whole new dataframe.

* **Tip:** You'll want to read the data dictionary for the Census data.
* **Tip:** The column names don't exactly match the data dictionary, but you can figure it out
* **Tip:** If you aren't sure which column is the actual census tract number, you can keep multiples for now

In [220]:
race.rename(columns={
    'Geo_TRACT':'Census_Tract', 
    'SE_A04001_001':'total', 
    'SE_A04001_003':'white', 
    'SE_A04001_004':'black', 
    'SE_A04001_010':'hispanic'
}, inplace=True)
race.head(5)

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,Geo_STATECE,Geo_STATE,Geo_COUNTY,Geo_COUSUB,Geo_PLACE,Geo_PLACESE,Census_Tract,Geo_BLKGRP,Geo_CONCIT,Geo_AIANHH,Geo_AIANHHFP,Geo_AIHHTLI,Geo_AITSCE,Geo_AITS,Geo_ANRC,Geo_CBSA,Geo_CSA,Geo_METDIV,Geo_MACC,Geo_MEMI,Geo_NECTA,Geo_CNECTA,Geo_NECTADIV,Geo_UA,Geo_UACP,Geo_CDCURR,Geo_SLDU,Geo_SLDL,Geo_VTD,Geo_ZCTA3,Geo_ZCTA5,Geo_SUBMCD,Geo_SDELM,Geo_SDSEC,Geo_SDUNI,Geo_UR,Geo_PCI,Geo_TAZ,Geo_UGA,Geo_BTTR,Geo_BTBG,Geo_PUMA5,Geo_PUMA1,total,SE_A04001_002,white,black,SE_A04001_005,SE_A04001_006,SE_A04001_007,SE_A04001_008,SE_A04001_009,hispanic,SE_A04001_011,SE_A04001_012,SE_A04001_013,SE_A04001_014,SE_A04001_015,SE_A04001_016,SE_A04001_017
0,55079000101,14000US55079000101,"Census Tract 1.01, Milwaukee County, Wisconsin","Census Tract 1.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4717,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5306,5015,1343,3567,13,70,0,16,6,291,37,0,0,0,0,254,0
1,55079000102,14000US55079000102,"Census Tract 1.02, Milwaukee County, Wisconsin","Census Tract 1.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4718,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3648,3549,1034,2195,36,118,0,0,166,99,67,32,0,0,0,0,0
2,55079000201,14000US55079000201,"Census Tract 2.01, Milwaukee County, Wisconsin","Census Tract 2.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4719,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4791,4019,1185,2657,0,102,0,0,75,772,120,0,0,0,0,613,39
3,55079000202,14000US55079000202,"Census Tract 2.02, Milwaukee County, Wisconsin","Census Tract 2.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4720,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6324,6148,2687,2810,87,390,12,0,162,176,113,0,0,0,0,63,0
4,55079000301,14000US55079000301,"Census Tract 3.01, Milwaukee County, Wisconsin","Census Tract 3.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4721,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339,1339,1005,220,0,90,0,10,14,0,0,0,0,0,0,0,0


In [225]:
race[['Census_Tract', 'total', 'white', 'black', 'hispanic']]
race['percent_white'] = race['white']/race['total']
race['percent_black'] = race['black']/race['total']
race['percent_hispanic'] = race['hispanic']/race['total']
race['percent_color'] = 1 - (race['white']/race['total'])
race

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,Geo_STATECE,Geo_STATE,Geo_COUNTY,Geo_COUSUB,Geo_PLACE,Geo_PLACESE,Census_Tract,Geo_BLKGRP,Geo_CONCIT,Geo_AIANHH,Geo_AIANHHFP,Geo_AIHHTLI,Geo_AITSCE,Geo_AITS,Geo_ANRC,Geo_CBSA,Geo_CSA,Geo_METDIV,Geo_MACC,Geo_MEMI,Geo_NECTA,Geo_CNECTA,Geo_NECTADIV,Geo_UA,Geo_UACP,Geo_CDCURR,Geo_SLDU,Geo_SLDL,Geo_VTD,Geo_ZCTA3,Geo_ZCTA5,Geo_SUBMCD,Geo_SDELM,Geo_SDSEC,Geo_SDUNI,Geo_UR,Geo_PCI,Geo_TAZ,Geo_UGA,Geo_BTTR,Geo_BTBG,Geo_PUMA5,Geo_PUMA1,total,SE_A04001_002,white,black,SE_A04001_005,SE_A04001_006,SE_A04001_007,SE_A04001_008,SE_A04001_009,hispanic,SE_A04001_011,SE_A04001_012,SE_A04001_013,SE_A04001_014,SE_A04001_015,SE_A04001_016,SE_A04001_017,percent_white,percent_black,percent_hispanic,percent_color
0,55079000101,14000US55079000101,"Census Tract 1.01, Milwaukee County, Wisconsin","Census Tract 1.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4717,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5306,5015,1343,3567,13,70,0,16,6,291,37,0,0,0,0,254,0,0.253110,0.672258,0.054844,0.746890
1,55079000102,14000US55079000102,"Census Tract 1.02, Milwaukee County, Wisconsin","Census Tract 1.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4718,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3648,3549,1034,2195,36,118,0,0,166,99,67,32,0,0,0,0,0,0.283443,0.601700,0.027138,0.716557
2,55079000201,14000US55079000201,"Census Tract 2.01, Milwaukee County, Wisconsin","Census Tract 2.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4719,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4791,4019,1185,2657,0,102,0,0,75,772,120,0,0,0,0,613,39,0.247339,0.554582,0.161135,0.752661
3,55079000202,14000US55079000202,"Census Tract 2.02, Milwaukee County, Wisconsin","Census Tract 2.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4720,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6324,6148,2687,2810,87,390,12,0,162,176,113,0,0,0,0,63,0,0.424889,0.444339,0.027830,0.575111
4,55079000301,14000US55079000301,"Census Tract 3.01, Milwaukee County, Wisconsin","Census Tract 3.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4721,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339,1339,1005,220,0,90,0,10,14,0,0,0,0,0,0,0,0,0.750560,0.164302,0.000000,0.249440
5,55079000302,14000US55079000302,"Census Tract 3.02, Milwaukee County, Wisconsin","Census Tract 3.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4722,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3091,2946,448,2363,11,40,0,0,84,145,74,25,0,0,0,34,12,0.144937,0.764478,0.046910,0.855063
6,55079000303,14000US55079000303,"Census Tract 3.03, Milwaukee County, Wisconsin","Census Tract 3.03, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4723,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1980,864,941,0,77,0,0,98,49,25,0,0,0,0,24,0,0.425826,0.463775,0.024150,0.574174
7,55079000304,14000US55079000304,"Census Tract 3.04, Milwaukee County, Wisconsin","Census Tract 3.04, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4724,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# <span style="color:red">Incomplete</span>

**I need to finish the rest of this**

# Combine this census data with your street address data

Create a new dataframe my merging your street addresses with your census data. It would be nice to marge on census tract code, but your addresses data does **not** have a census tract on them! **This is where things get a little crazy.**

### Adding census tract codes to addresses

We went ahead and geocoded many addresses in Milwaukee to create `addresses_geocoded.csv`. You'll need to add a new column using QGIS that adds in the address's census tract.

The QGIS command **Join attributes by location** will merge datasets that geographically overlap each other. If you have one layer that's a list of lat/lon points and one layer that's a shapefile with census tract information, **join attributes by location** can create a new layer of lat/lon points that also has census tract information.

You can export the result by right clicking the layer, selecting **Export As** and making sure the output is another CSV.

### Merging

Now you'll have three datasets:

* Addresses and pothole fill ties
* Census data according with census tract codes
* Addresses with census tract codes

You'll need to merge them each together to create one whole dataset.

# Linear regression

Using the `statsmodels` package, run a linear regression to find the coefficient relating percent minority and pothole fill times.

Translate that into the form **"every X percentage point change in the minority population translates to a Y change in pot hole fill times"**

Do you feel comfortable that someone can understand that? Can you reword it to make it more easily understandable?

# Other methods of explanation

While the regression is technically correct, it just does't sound very nice. What other options do we have?

## What's the average wait to fill a pothole between majority-white and majority-minority census tracts?

You'll need to create a new column to specify whether the census tract is majority White or not.

## How does the average wait time to fill a pothole change as more minorities live in an area?

* **Tip:** Use `.cut` or `.qcut` to split the percent minority (or white) into a few different bins.

# Bonus: Income

`R12216226_SL140.csv` contains income data for each census tract in Wisconsin. Add it into your analysis.

If you run a multivariate regression also including income, how does this change things?